In [4]:
## Install the package if needed:
#install.packages('geosphere', repos='http://cran.us.r-project.org')

## Load the relevent packages
library(ncdf4)  # For reading in the NCEP wind fields
#library(R.matlab)  # If you need to read in matlab .mat files
library(openxlsx)  # If you need to read in .xlsx files
#library(rNOMADS)  # For reading grib2 data files (NOMADS data for instance)
#library(rGDAL)  #
library(RColorBrewer)
library(compiler)  # required for JIT (below)
#library(lattice)
#library(geosphere)
library(data.table)

## Enable compilation (speed gain?)
enableJIT(3)

## Helper function for converting the date time stamps.
conv_excel_time = function(x, tz='GMT') {
    as.POSIXct(as.Date(x, origin="1899-12-30", tz=tz))
}

conv_roms_time = function(x, tz='UTC') {
    as.POSIXct(x, origin="1900-01-01")
}

get.qual.pal = function(n=100, pal='Accent') {
    colorRampPalette(brewer.pal(8, pal))(n)
}

get.seq.pal = function(n=100, pal='YlOrRd') {
    colorRampPalette(rev(brewer.pal(9, pal)))(n)
}

get.div.pal = function(n=100, pal='Spectral') {
    colorRampPalette(rev(brewer.pal(11, pal)))(n)
}

make.div.pal = function(x=100, n, pal='Spectral') {
    get.div.pal(n, pal=pal)[as.numeric(cut(x, breaks = n))]
}

[1] 3

In [5]:
input.dir = 'Raw Data/O2-optode MIMS/'

file.list = list.files(input.dir)
length(file.list)

[1] 44

In [70]:
process.file = function(input.dir, file) {
    data = data.frame(Date = '', Time = '', Rel.Time = '',
                      Oxy.ch1 = 0, Oxy.ch2 = 0, Oxy.ch3 = 0, Oxy.ch4 = 0,
                      Temp.ch1 = 0, Temp.ch2 = 0, Temp.ch3 = 0, Temp.ch4 = 0, Pressure = 0, stringsAsFactors = FALSE)
    temp = fread(paste0(input.dir, file.list[1]), skip = 20)[,c(1:3, 5:13)]
    colnames(temp) = colnames(data)
    data = rbind(data, temp)
    data = data[-1,]
    data = as.data.frame(data)
    data$Time = add.time(data$Date, data$Time)
    data
}

add.time = function(dates, times) {
    ans = as.POSIXct('5/16/2017 14:07:10', format = '%m/%d/%Y %H:%M:%S', tz = 'GMT')
    for (i in 1:length(dates)) {
        ans = c(ans, as.POSIXct(paste(dates[i], times[i]), format = '%m/%d/%Y %H:%M:%S', tz = 'GMT'))
    }
    ans = ans[-1]
    ans
}

avg.data = function(data, N = 120) {
    i = 1
    
    for (j in 3:ncol(data)) {
        data[,j] = as.numeric(data[,j])
    }
    
    while (i < nrow(data)) {
        dt = as.numeric(difftime(data$Time, data$Time[i], unit = 'secs'))
        
        l = which(dt >= 0 & dt < N)
        
        temp = apply(data[l, 3:ncol(data)], 2, function(x) {mean(x, na.rm = TRUE)})
        
        data[i, 3:ncol(data)] = temp
        
        if (length(l) > 1) {
            l = l[l != i]
            if (length(l) > 0) {
                data = data[-l,]
            }
        }
        i = i + 1
    }
    data
}

In [ ]:
for (i in file.list[2:length(file.list)]) {
    temp = 0
    try({temp = avg.data(process.file(input.dir, i))})
    write.xlsx(temp, paste0(input.dir, i, '-parsed.xlsx'))
}

Warning message in fread(paste0(input.dir, file.list[1]), skip = 20):
"Bumped column 5 to type character on data row 5599, field contains '---'. Coercing previously read values in this column from logical, integer or numeric back to character which may not be lossless; e.g., if '00' and '000' occurred before they will now be just '0', and there may be inconsistencies with treatment of ',,' and ',NA,' too (if they occurred in this column before the bump). If this matters please rerun and set 'colClasses' to 'character' for this column. Please note that column type detection uses a sample of 1,000 rows (100 rows at 10 points) so hopefully this message should be very rare. If reporting to datatable-help, please rerun and include the output from verbose=TRUE."

In [72]:
str(temp)

'data.frame':	70560 obs. of  12 variables:
 $ Date    : chr  "5/16/2017" "5/16/2017" "5/16/2017" "5/16/2017" ...
 $ Time    : POSIXct, format: "2017-05-16 10:07:10" "2017-05-16 10:07:11" ...
 $ Rel.Time: chr  "0" "0.94" "1.88" "2.83" ...
 $ Oxy.ch1 : chr  "19.59200" "19.43400" "19.59100" "19.55400" ...
 $ Oxy.ch2 : chr  "---" "---" "---" "---" ...
 $ Oxy.ch3 : chr  "---" "---" "---" "---" ...
 $ Oxy.ch4 : chr  "---" "---" "---" "---" ...
 $ Temp.ch1: num  20 20 20 20 20 20 20 20 20 20 ...
 $ Temp.ch2: num  20 20 20 20 20 20 20 20 20 20 ...
 $ Temp.ch3: num  20 20 20 20 20 20 20 20 20 20 ...
 $ Temp.ch4: num  20 20 20 20 20 20 20 20 20 20 ...
 $ Pressure: num  1012 1012 1012 1012 1012 ...


In [59]:
l

ERROR: Error in eval(expr, envir, enclos): object 'l' not found
